# Sentence Similarity
Sentence similarity involves converting sentences into high-dimensional vector representations and then calculating the similarity between these vectors. 

This is typically done using pre-trained models that generate sentence embeddings. These embeddings can be used to calculate the similarity between sentences using cosine similarity or other distance metrics.

## Single Sentence Example

In [20]:
from sentence_transformers import SentenceTransformer, util

In [21]:
# Load an example pre-trained sentence transformer models
model_miniLM = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/Users/nitingupta/miniconda3/envs/auto-plan/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
# Intent questions for each category
intents = {
    1: ["Why is action A not used in the plan?"],
    2: ["Why is action A used in the plan?"],
    3: ["Why is action A used rather than action B?"]
}

# Encode example questions
intent_embeddings_miniLM = {k: model_miniLM.encode(v) for k, v in intents.items()}

In [23]:
def categorize_question_similarity(question, model, intent_embeddings):
    question_embedding = model.encode(question)  # Encode the input question

    # Compute similarity scores
    scores = {k: util.pytorch_cos_sim(question_embedding, v).item() for k, v in intent_embeddings.items()}
    
    # Find the category with the highest similarity score
    return max(scores, key=scores.get)

In [24]:
question = "Why is action A used rather than action B?"
category = categorize_question_similarity(question, model_miniLM, intent_embeddings_miniLM)
print(f"The question belongs to category: {category}")

The question belongs to category: 3


<br>

## Evaluate Different Models on the Dataset

In [25]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

df = pd.read_csv('./data/combined_dataset.csv')
print(f"Number of rows in the dataset: {df.shape[0]}")
df.head()

Number of rows in the dataset: 346


,text,label
0,Why was action A excluded from the plan?,1
1,What were the reasons for omitting action A fr...,1
2,Can you explain why action A was not considere...,1
3,Why didn't the plan include action A?,1
4,What is the rationale for not using action A i...,1


In [26]:
def text_similarity_from_model(model_name, df):
    model = SentenceTransformer(model_name)
    intent_embeddings = {k: model.encode(v) for k, v in intents.items()}

    # Apply the classification function to the dataset
    df['predicted_label'] = df['text'].apply(categorize_question_similarity, model=model, intent_embeddings=intent_embeddings)
    
    # Evaluate the results
    y_true = df['label']
    y_pred = df['predicted_label']
    
    return classification_report(y_true, y_pred), accuracy_score(y_true, y_pred)

In [27]:
class_report, accuracy = text_similarity_from_model("paraphrase-MiniLM-L6-v2", df)
print(class_report)
print(f"Accuracy: {accuracy:.2f}")

/Users/nitingupta/miniconda3/envs/auto-plan/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.90      0.81      0.85       116
           2       0.52      1.00      0.68       115
           3       1.00      0.17      0.28       115

    accuracy                           0.66       346
   macro avg       0.81      0.66      0.61       346
weighted avg       0.81      0.66      0.61       346

Accuracy: 0.66


In [28]:
class_report, accuracy = text_similarity_from_model("all-MiniLM-L12-v2", df)
print(class_report)
print(f"Accuracy: {accuracy:.2f}")

/Users/nitingupta/miniconda3/envs/auto-plan/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.72      0.87      0.79       116
           2       0.65      0.83      0.73       115
           3       0.83      0.43      0.56       115

    accuracy                           0.71       346
   macro avg       0.73      0.71      0.69       346
weighted avg       0.73      0.71      0.69       346

Accuracy: 0.71
